# Download dataset

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kritikseth/fruit-and-vegetable-image-recognition")

print("Path to dataset files:", path)

Mounting files to /kaggle/input/fruit-and-vegetable-image-recognition...
Path to dataset files: /kaggle/input/fruit-and-vegetable-image-recognition


# Fine tuning model

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
import os

# -----------------------------
# 1. Basic config
# -----------------------------
DATA_DIR = "/kaggle/input/fruit-and-vegetable-image-recognition"          # root folder
TRAIN_DIR = os.path.join(DATA_DIR, "train")
VAL_DIR   = os.path.join(DATA_DIR, "validation")
TEST_DIR  = os.path.join(DATA_DIR, "test")   # optional, for final eval

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42

# -----------------------------
# 2. Load datasets
# -----------------------------
train_ds = keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=SEED,
)

val_ds = keras.utils.image_dataset_from_directory(
    VAL_DIR,
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False,
)

test_ds = keras.utils.image_dataset_from_directory(
    TEST_DIR,
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False,
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)
print("Num classes:", num_classes)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)
test_ds  = test_ds.prefetch(AUTOTUNE)

# -----------------------------
# 3. Data augmentation pipeline
# -----------------------------
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.05),
        layers.RandomZoom(0.1),
        layers.RandomTranslation(0.05, 0.05),
    ],
    name="data_augmentation",
)

# -----------------------------
# 4. Build the base MobileNetV2
# -----------------------------
base_model = MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights="imagenet",
)

# Freeze base model for the warm-up phase
base_model.trainable = False

# -----------------------------
# 5. Build the full model
# -----------------------------
inputs = keras.Input(shape=IMG_SIZE + (3,), name="input_image")

# (a) Data augmentation
x = data_augmentation(inputs)

# (b) Rescale pixels to [0, 1]
x = layers.Rescaling(1.0 / 255)(x)

# (c) Pass through base MobileNetV2
x = base_model(x, training=False)

# (d) Pooling + classification head
x = layers.GlobalAveragePooling2D(name="global_avg_pool")(x)
x = layers.Dropout(0.3, name="dropout")(x)
outputs = layers.Dense(num_classes, activation="softmax", name="predictions")(x)

model = keras.Model(inputs, outputs, name="fruits_veggies_mobilenetv2")

model.summary()

# -----------------------------
# 6. Compile (warm-up training)
# -----------------------------
initial_lr = 1e-3
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=initial_lr),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

initial_epochs = 5

history_warmup = model.fit(
    train_ds,
    epochs=initial_epochs,
    validation_data=val_ds,
)

# -----------------------------
# 7. Fine-tune (unfreeze some layers)
# -----------------------------
# Unfreeze the top part of MobileNetV2 for fine-tuning
base_model.trainable = True

# Optionally: freeze earlier layers, fine-tune only last N
fine_tune_at = 100  # unfreeze from this layer onwards (tune as needed)

for i, layer in enumerate(base_model.layers):
    if i < fine_tune_at:
        layer.trainable = False
    else:
        layer.trainable = True

# Recompile with a much lower LR
fine_tune_lr = 1e-5
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=fine_tune_lr),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

history_finetune = model.fit(
    train_ds,
    epochs=total_epochs,
    initial_epoch=history_warmup.epoch[-1] + 1,
    validation_data=val_ds,
)

# -----------------------------
# 8. Evaluate and save
# -----------------------------
loss, acc = model.evaluate(val_ds)
print(f"Validation accuracy after fine-tuning: {acc:.4f}")

model.save("mobilenetv2_fruits_veggies_finetuned.h5")
print("Model saved as mobilenetv2_fruits_veggies_finetuned.h5")


Found 3115 files belonging to 36 classes.
Found 351 files belonging to 36 classes.
Found 359 files belonging to 36 classes.
Classes: ['apple', 'banana', 'beetroot', 'bell pepper', 'cabbage', 'capsicum', 'carrot', 'cauliflower', 'chilli pepper', 'corn', 'cucumber', 'eggplant', 'garlic', 'ginger', 'grapes', 'jalepeno', 'kiwi', 'lemon', 'lettuce', 'mango', 'onion', 'orange', 'paprika', 'pear', 'peas', 'pineapple', 'pomegranate', 'potato', 'raddish', 'soy beans', 'spinach', 'sweetcorn', 'sweetpotato', 'tomato', 'turnip', 'watermelon']
Num classes: 36
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "fruits_veggies_mobilenetv2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_avg_pool                 │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 36)             │        46,116 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,304,100 (8.79 MB)

 Trainable params: 46,116 (180.14 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/5


I0000 00:00:1763569531.656811     122 cuda_dnn.cc:529] Loaded cuDNN version 90300


98/98 ━━━━━━━━━━━━━━━━━━━━ 39s 312ms/step - accuracy: 0.2448 - loss: 2.9245 - val_accuracy: 0.8063 - val_loss: 0.7406
Epoch 2/5
98/98 ━━━━━━━━━━━━━━━━━━━━ 26s 268ms/step - accuracy: 0.7032 - loss: 1.0229 - val_accuracy: 0.8746 - val_loss: 0.4781
Epoch 3/5
98/98 ━━━━━━━━━━━━━━━━━━━━ 26s 261ms/step - accuracy: 0.7940 - loss: 0.7449 - val_accuracy: 0.9003 - val_loss: 0.3736
Epoch 4/5
98/98 ━━━━━━━━━━━━━━━━━━━━ 26s 265ms/step - accuracy: 0.8272 - loss: 0.6013 - val_accuracy: 0.9088 - val_loss: 0.3260
Epoch 5/5
98/98 ━━━━━━━━━━━━━━━━━━━━ 26s 268ms/step - accuracy: 0.8249 - loss: 0.5527 - val_accuracy: 0.9031 - val_loss: 0.2980
Epoch 6/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 41s 279ms/step - accuracy: 0.6339 - loss: 1.2347 - val_accuracy: 0.9174 - val_loss: 0.2757
Epoch 7/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 269ms/step - accuracy: 0.7430 - loss: 0.8656 - val_accuracy: 0.9202 - val_loss: 0.2704
Epoch 8/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 27s 272ms/step - accuracy: 0.7807 - loss: 0.7338 - val_accuracy: 0.9202 - 

Validation accuracy after fine-tuning: 0.9402
Model saved as mobilenetv2_fruits_veggies_finetuned.h5
